### Homework 2

이번 과제에서는 pandas datareader를 이용하여 주가 데이터를 이용해 실습을 한다

#### 1. 데이터 불러오기

아래 라이브러리를 활용하여 2020년 1월 1일부터 2023년 6월 31일까지의 어느 한 주식 종목의 데이터를 불러오라

In [ ]:
import pandas_datareader as pdr
import datetime
import yfinance as yf

In [ ]:
start_date = '2020-01-01'
end_date = '2023-07-01'

apple = yf.download('AAPL', start = start_date, end = end_date)
apple

#### 2. AR model

준비된 데이터에 대해 AR모형의 차수를 결정하고 AR model을 적합화(fit)해 보자. 모형을 가장 잘 적합화하기 위해서는 차수를 어떻게 정해야 하는가?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

In [ ]:
ret = np.log(apple.Close) - np.log(apple.Close.shift())
ret = ret.dropna()
ret

AR 차수 구하기

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
plot_pacf(np.log(apple.Close), auto_ylims = True, lags = 30, title = 'PACF of Tesla log Index', ax = ax)
plt.xlabel('Time lags', fontsize = 10)
plt.title('Partial Auto Correlation Function', fontsize = 15, fontfamily = 'Serif')
plt.show()

In [ ]:
model = sm.tsa.ARIMA(np.log(apple.Close), order = (1,0,0), trend = 'c').fit(method = 'innovations_mle')
print(model.summary())

In [ ]:
model.plot_diagnostics(figsize = (14,7))
plt.show()

#### 3. MA model

준비된 데이터에 대해 MA모형의 차수를 결정하고 MA model을 적합화(fit)해 보자. 모형을 가장 잘 적합화하기 위해서는 차수를 어떻게 정해야 하는가? 그리고 MA model을 사용하기 위해서는 어떤 데이터를 사용해야 하는가?

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
plot_acf(ret, auto_ylims = True, lags = 30, title = 'ACF of Tesla return' , ax = ax)
plt.xlabel('Time lags', fontsize = 10)
plt.title('Auto Correlation Function', fontsize = 15, fontfamily = 'Serif')
plt.show()

In [ ]:
model = sm.tsa.ARIMA(ret, order = (0,0,1), trend = 'c').fit(method = 'innovations_mle')
print(model.summary())

In [ ]:
model.plot_diagnostics(figsize = (14,7))
plt.show()

#### 4. ARMA model

준비된 데이터에 대해 ARMA모형의 차수(p,q)를 결정하고 ARMA model을 적합화(fit)해 보자. 모형을 가장 잘 적합화하기 위해서는 차수를 어떻게 정해야 하는가? 그리고 ARMA model을 사용하기 위해서는 어떤 데이터를 사용해야 하는가?

In [ ]:
model = sm.tsa.ARIMA(np.log(apple.Close), order = (1,0,1), trend = 'c').fit(method = 'innovations_mle')
print(model.summary())

In [ ]:
import seaborn as sns
fig, ax1 = plt.subplots(figsize=(8,5), dpi=100)
sns.distplot(np.random.normal(size=100000), kde = True, ax = ax1, label = 'Normal Distribution')
sns.distplot(np.log(model.resid) - np.log(model.resid).mean() ,
             kde = True, rug = False , ax = ax1, label = 'Log Residual Distribution')
plt.legend()
plt.title('Residual vs. Normal', fontsize = 15, fontfamily = 'Serif')

In [ ]:
model.plot_diagnostics(figsize = (14,7))
plt.show()

#### 5. Prediction

1~4번을 통해 구한 모형으로부터 20기간 예측을 실행하라. 어떤 결과값이 나오는가? 실제값과 비교하였을 때, 예측은 정확한가? 추정된 값은 효율적인 추정량이고, 최소 예측 분산을 가지고 있는가?

In [ ]:
forecast_steps = 20
model.forecast(steps = forecast_steps)

In [ ]:
start_date = '2023-06-30'
end_date = '2023-07-30'

apple_new = yf.download('AAPL', start = start_date, end = end_date)

In [ ]:
import pandas as pd
plt.figure(figsize = (10,4))
plt.plot(model.fittedvalues[-10:], label = 'train')
plt.plot(pd.date_range(start = '2023-06-30', periods = forecast_steps, freq='B'),
         model.forecast(steps = forecast_steps), label = 'predicted')
plt.plot(np.log(apple_new.Close), label = 'test')
plt.xticks(rotation = 45, ha = 'right')
plt.legend(loc = 'center left', bbox_to_anchor = (1, 0.5))

#### 6. Recurrent Neural Network

<center><img src = "https://images.velog.io/images/yuns_u/post/ccbb28ea-fa08-4d23-804e-419e6f578e4b/image.png" alt="My Image"></center>

**RNN(Recurrent Neural Network)** 은 딥러닝 기법 중 하나로, 시퀀스 데이터를 처리하는 데 특화된 인공신경망 구조이다. 자연어 처리, 음성 인식, 시계열 데이터 등 다양한 시퀀스 데이터에 대한 분석과 예측에 매우 유용하게 사용된다고 알려져 있다. RNN에 관한 자세한 내용은 아래 링크를 통해 익히도록 하자

https://pytorch.org/docs/stable/generated/torch.nn.RNN.html

1개의 hidden layer, $H=1$, activation function은 없고 입력 벡터의 차원 $P=1$인 간단한 경우의 $\mbox{RNN}$을 추가로 고려하자. 추가로 $W_z^{(1)} = \phi_z$, $|\phi_z|<1$, $W_x^{(1)} = \phi_x$, $W_y = 1$, $b_h = 0$, $b_y = \mu$를 가정하자. 그러면 $f_{W^{(1)},b^{(1)}}^{(1)}(\chi_t)$가 자기회귀 계수가 기하급수적으로 감쇠하는, 즉 $\phi_i = \phi_x \phi_z^{i-1}$인 $p$차의 자기회귀 $\mbox{AR}(p)$모델임을 보일 수 있다

$$z_{t-p} = \phi_x x_{t-p}$$
$$z_{t-T+2} = \phi_z z_{t-T+1} + \phi_x x_{x-T+2}$$
$$\cdots = \cdots$$
$$z_{t-1} = \phi_z z_{t-2} + \phi_x x_{t-1}$$
$$\hat x_t = z_{t-1} + \mu$$

위 조건에서 다음이 성립한다

$$\hat x_t = \mu + \phi_x(L + \phi_z L^2 + \cdots + \phi_z^{p-1} L^p)[x_t] = \mu + \sum_{i=1} \phi_i x_{t-i}$$

위 조건을 참고하여, 아래 라이브러리를 사용하여 Activation function을 사용하지 않고 은닉층이 한 개인 간단한 RNN모형을 사용하여 2번의 과정을 다시 반복하라.

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense
from keras.regularizers import l1, l2

scaling

In [ ]:
min_val = np.min(apple.Close)
max_val = np.max(apple.Close)
scaled_data = (apple.Close - min_val) / (max_val - min_val)

In [ ]:
X = scaled_data[1:]
y = scaled_data.shift(1).dropna()

train - test split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = False)

Simple RNN models

In [ ]:
model = Sequential()
model.add(SimpleRNN(units = 1, 
                    input_shape = (X_train.shape[0], 1), 
                    kernel_regularizer = l1(0), 
                    unroll = True, 
                    stateful = False,
                    activation = None))
model.add(Dense(1, kernel_regularizer = l1(0)))
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
model.fit(X_train, y_train, epochs = 100, verbose = 0)

In [ ]:
predicted_train = model.predict(X_train)
predicted_test = model.predict(X_test)

In [ ]:
test = pd.Series(predicted_test.reshape(-1), index = X_test.index)

In [ ]:
plt.figure(figsize = (10, 4))
plt.plot(X_train, label = 'Train data')
plt.plot(test, label = 'Predicted data')
plt.plot(X_test, label = 'Test data')
plt.grid(False)
plt.legend(loc = 'center left', bbox_to_anchor = (1, 0.5))
plt.title('AR model prediction with simple RNN', fontsize = 12, fontfamily = 'Serif')